In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format='{:.2f}'.format
plt.style.use('seaborn')
import sklearn as skl
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import io
import shutil

In [ ]:
import warnings
warnings.filterwarnings("ignore")

class MyDF(pd.DataFrame):
    @staticmethod
    def summarize_fin2(fin2):
        no=3
        first=fin2[:3]
        second=fin2[6:]
        second=second.split()
        second=[x[:no] for x in second]
        second=''.join(second)
        return '_'.join([first,second])
    
    @staticmethod
    def is_dhic(fund_type):
        return 1 if fund_type[:2] in ['DA','DO'] else 0
    
    @staticmethod
    def account_type(row):
        first=row['fin21']
        first=0 if first==9 else first
        return first
    
    @staticmethod
    def add_features(df):
        inpt=input('Add features?  y/n')
        inpt=inpt.upper()
        if not inpt=='Y': return df
        df['account1']=df['account'].str.slice(0,1).astype(int)
        df['fin21']=df['fin2'].str.slice(0,1).astype(int)
        df['utp']=df['account'].str.slice(7,8).astype(int)
        df['dhic']=df['fund_type'].apply(df.is_dhic)
        df['acct_type']=df.apply(df.account_type,axis=1)
        return df
        
    @staticmethod
    def eliminate_zeros(df):
        return df[df['amount']!=0.0]
    
    @staticmethod
    def sort(df):
        df.sort_values(by=['fund','account','trant'],inplace=True)
    
    @staticmethod
    def copy_export_to_model():
        inpt=input('Copy export to model?  y/n')
        inpt=inpt.upper()
        if not inpt=='Y': return
        source = 'export.xlsx'
        destination = "model.xlsx"
        dest = shutil.copyfile(source, destination)
        print('copying is completed...')
        
    def __init__(self, *args, **kwargs):
        super(MyDF,  self).__init__(*args, **kwargs)
        self.ordinal_encoders={}

    @property
    def _constructor(self):
        return MyDF
    
#     def make_hints(self):
#         self['hints']=self['fin2'].apply(self.__class__.summarize_fin2)
    
    def get_info_df(self):
        buffer = io.StringIO()
        self.info(buf=buffer)
        lines=buffer.getvalue().splitlines()
        df=(pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
               .drop('Count',axis=1)
               .rename(columns={'Non-Null':'Non-Null Count'}))
#         df=df[df['Dtype']=='object']  #filter for strings
        print(df)
    
    def get_string_cols(self):
        buffer = io.StringIO()
        self.info(buf=buffer)
        lines=buffer.getvalue().splitlines()
        lines=lines[5:-2]
        lines=[x.split() for x in lines]
        lines=filter(lambda x:x[-1]=='object',lines)
        lines=[x[1] for x in lines]
        return lines
#         print(lines)
#         df=(pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
#                .drop('Count',axis=1)
#                .rename(columns={'Non-Null':'Non-Null Count'}))
# #         df=df[df['Dtype']=='object']
#         print(df)
#     def get_encoders(self):
#         return self.ordinal_encoders
    
    def encode(self):
        print('\nencoding...')
        # encode the columns
        encoders=self.ordinal_encoders
        for col in  self.get_string_cols():
            enc=preprocessing.OrdinalEncoder()
            encoders[col]=enc
#             print('transforming {}...'.format(col))
            self[col]=enc.fit_transform(self[[col]])
            
    def decode(self):
        print('\ndecoding...')
        #decode the columns
        encoders=self.ordinal_encoders
        columns=list(encoders.keys())
        for col in columns:
            enc=encoders[col]
            self[col]=enc.inverse_transform(self[[col]])
            encoders.pop(col)
    
    def columns_X_y(self,ycol):
        Xcols=list(self.columns[:])
#         print(cols)
        Xcols.remove(ycol)
        print('X columns are {}'.format(str(Xcols)))
        print('y column is {}'.format(ycol))
        X=self[Xcols]
        y=self[ycol]
        return X,y,Xcols,ycol
    
#     def check_columns(self,dfmodel):
#         model_cols=list(dfmodel.columns)
#         df_cols=list(self.columns)
#         if model_cols!=df_cols:
#             print('model columns: %s'%str(model_cols)+' does NOT equal '+'df columns: %s'%str(df_cols))
#             return False
#         else:
#             return True
    
    def get_clf(self,fpth='model.xlsx',ycol='target'):
        print('getting clf, using file {}...'.format(fpth))
        dfmodel=pd.read_excel(fpth)
        dfmodel=MyDF(dfmodel)
        print('droping NaN')
        dfmodel.dropna(how='any',axis=0,inplace=True)
#         if not self.check_columns(dfmodel): return
        print('encoding the model...')
        dfmodel.encode()
        X,y,Xcols,ycol=dfmodel.columns_X_y(ycol)
#         clf=DecisionTreeClassifier()
        clf=RandomForestClassifier(n_estimators=50)
        print('fitting the classifier...')
        clf.fit(X,y)
        self.clf_data=[clf,X,y,Xcols,ycol]
        self.ordinal_encoders[ycol]=dfmodel.ordinal_encoders[ycol]
#         return clf
    
    def predict(self,fpth='model.xlsx',ycol='target'):
        print('predicting...')
        self.get_clf(fpth,ycol)
        clf,X,y,Xcols,ycol=self.clf_data
        print('columns presented to fit: ',Xcols,[ycol])
#         print(ycol)
        self.drop(ycol,axis=1,inplace=True)
        print('columns presenting to predict: ',self.columns)
        predictions=clf.predict(df)
        self[ycol]=predictions
        self.decode()
        self.head(100)

    def export(self,fpth='export.xlsx'):
        self.to_excel(fpth,index=False)
    
    def export_stub(self,fpth='stub.xlsx'):
        inpt=input('Export stub?  y/n')
        inpt=inpt.upper()
        if not inpt=='Y': return
        self['target']=''
        self.to_excel(fpth,index=False)
    


In [ ]:
MyDF.copy_export_to_model()

In [ ]:
fpth='output_2022.xlsx'
fpth='model.xlsx'
df=pd.read_excel(fpth)
df=MyDF(df)
df=MyDF.eliminate_zeros(df)
df=MyDF.add_features(df)
# df['target'].fillna('0', inplace=True)
# df['target'].dropna(how='any',axis=1)
df.head()

In [ ]:
MyDF.sort(df)
df.export_stub()
df.head()

In [ ]:
df.encode()
df.head()

In [ ]:
df.predict()

In [ ]:
MyDF.sort(df)
df.export()